<a href="https://colab.research.google.com/github/ryujiuno0323/TFL_Maker/blob/master/FlickrAPI%E3%81%A7%E7%94%BB%E5%83%8F%E3%82%92%E5%8F%8E%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 今回したいこと
- 画像データをWEB上から集めて、PC上に画像データとして保存する．







# Webから画像を収集する
参考<br>
https://toyo-interest.com/python/web-application-%E7%AC%AC%EF%BC%93%E5%9B%9E%E3%80%80%E3%81%AF%E3%81%98%E3%82%81%E3%81%A6%E3%81%AEweb%E3%82%A2%E3%83%97%E3%83%AA/

http://ykubot.com/2017/11/05/flickr-api/

フリッカーの日本語サイト：<br>
https://www.flickr.com/groups/japanese/














In [0]:
# TensorFlowの最新バージョンへ更新
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.2.0)


In [0]:
import os
import glob
import shutil
import random

from urllib import request as req
from urllib import error
from urllib import parse
import bs4

#＜1.データ抽出・保存＞iFlicerAPIでcrawler後Driveにデータ保存

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
p_temp=os.getcwd()
print(p_temp)
print(os.listdir(path=p_temp))

/content
['.config', 'drive', 'sample_data']


In [0]:
# FlickrAPIのインストール
!pip install flickrapi

     |████████████████████████████████| 61kB 3.1MB/s 


In [0]:
pip install retry

#FlicerAPIでのicrawler保存先


In [0]:
    #①icrawler 保存先path
    #path_app="/content/drive/My Drive/aiapp/"
    path_app="/content/drive/My Drive/aiapp/"

    #original_dir=path_app+"☆_data_0☆/"
    path_original = path_app+"class_3_clover_data_07/"
    #path_original = path_app+"class_3_Flicer_data_01/"
    original_dir = path_original
    print ("original_dir=path_original=")
    print (original_dir)

original_dir=path_original=
/content/drive/My Drive/aiapp/class_3_clover_data_07/


## 手順
主な手順としては、以下のようになります．

- step 1
  - FlickrのAPIを使って欲しい画像のタグ（dogとかcatとか）とURLを取得して、リスト化する
- step 2
  - リストのURLに従い、**urlretrieve**を使ってダウンロードしていく
  
  

In [0]:
"""
# 必要なライブラリのインポート
import os, time, sys, math
from flickrapi import FlickrAPI
from urllib.request import urlretrieve
import shutil

# APIキーの情報
#flickr_api_key = "f7e70239910909ab634b165235675395"
key = "f7e70239910909ab634b165235675395"
#secret_key = "5a8e19991c97f17e"
secret = "5a8e19991c97f17e"
wait_time = 0.5 # 待ち時間（0.5秒おきにDLを行う）

#--------- INPUT -------------------------------------------
# ダウンロードしたい名前をclassesに記述
classes = ["four_leaf_clover", "three_leaf_clover","autumn_leaves"]
#classes = ["three_leaf_clover","autumn_leaves"]

#classes = ["autumn_leaves"]
#classes = ["cheetah", "tiger", "leopard", "lynx", "lion",\
#           "jaguar", "white tiger", "cat"]
# DLしたい枚数
per_page = 500
#-----------------------------------------------------------

# 枚数によって場合分け
if per_page < 500:
    per_page = per_page
    n_page = 1
else:
    n_page = math.floor(per_page/500) + 1
    per_page = math.floor(per_page / n_page)


nb_class = len(classes) #クラス数取得
print("classes=",nb_class)


#os.mkdir("./images")
#%cd ./images

# 保存フォルダの作成：
for i in range(nb_class):
    dir_name = classes[i] 
    # 指定した名前のフォルダを作成する
    #dirpath = "./" + dir_name
    dirpath = original_dir  + dir_name
    #os.mkdir(dirpath)
    os.makedirs(dirpath, exist_ok=True)
    print(dirpath)

#-----------------------------------------
# per_data：DLしたい枚数
# safe_serch：1が安全らしい
# extras：url_qとライセンス情報（おそらくこれで十分）
#-----------------------------------------
    
    flickr = FlickrAPI(key, secret, format="parsed-json")
    for i in range(n_page):
        response = flickr.photos.search(
            text = dir_name,
            per_page = per_page,
            page = i + 1,
            media = "photos",
            sort = "relevance", # 関連純
            safe_search = 1,
            extras = "url_q, licence"
            )

        photos = response["photos"] # 
#-----------------------------------------
# ここまでで今からダウンロードするファイルの情報（id、URLなど）を取得してリスト化
# リストに従い、urlretrieveで1つずつ指定フォルダにダウンロードしていく
# urlretrieve("ダウンロードしたいものがあるURL", "保存先")
#
# 画像によってurl_qがない場合にエラーが発生して止まってしまうので、その場合は無視して次の画像に進むようにしている
#-----------------------------------------

        for i, photo in enumerate(photos["photo"]):
            try:
                url_q = photo["url_q"]
                filepath = dirpath + "/" + photo["id"] + ".jpg"
                if os.path.exists(filepath): continue
                urlretrieve(url_q, filepath)
                time.sleep(wait_time)
            except:
                continue


classes= 3
/content/drive/My Drive/aiapp/class_3_clover_data_07/four_leaf_clover


In [0]:

classes = ["four_leaf_clover", "three_leaf_clover","autumn_leaves"]
nb_class = len(classes) #クラス数取得
print("classes=",nb_class)


classes= 3


In [0]:
# 保存フォルダの内容確認：


for i in range(nb_class):
    dir_name = classes[i] 
    # 指定した名前のフォルダを作成する
    #dirpath = "./" + dir_name
    dirpath = original_dir  + dir_name
    #os.mkdir(dirpath)
    #os.makedirs(dirpath, exist_ok=True)
    #print(dirpath)

    path = dirpath
    print(path)
    files = os.listdir(path)  #ファイル名とディレクトリ名の両方のリストが返る
    count = len(files)  
    print(count)
    print(files)  

/content/drive/My Drive/aiapp/class_3_clover_data_07/four_leaf_clover
459
['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg', '000006.jpg', '000007.jpg', '000008.jpg', '000009.jpg', '000010.jpg', '000011.jpg', '000012.jpg', '000013.jpg', '000014.jpg', '000015.jpg', '000016.jpg', '000017.jpg', '000018.jpg', '000019.jpg', '000020.jpg', '000021.jpg', '000022.jpg', '000023.jpg', '000024.jpg', '000025.jpg', '000026.jpg', '000027.jpg', '000028.jpg', '000029.jpg', '000030.jpg', '000031.jpg', '000032.jpg', '000033.jpg', '000034.jpg', '000035.jpg', '000036.jpg', '000037.jpg', '000038.jpg', '000039.jpg', '000040.jpg', '000041.jpg', '000042.jpg', '000043.jpg', '000044.jpg', '000045.jpg', '000046.jpg', '000047.jpg', '000048.jpg', '000049.jpg', '000050.jpg', '000051.jpg', '000052.jpg', '000053.jpg', '000054.jpg', '000055.jpg', '000056.jpg', '000057.jpg', '000058.jpg', '000059.jpg', '000060.jpg', '000061.jpg', '000062.jpg', '000063.jpg', '000064.jpg', '000065.jpg', '000066.jpg', '

In [0]:
p_temp= original_dir
print(p_temp)
print(os.listdir(path=p_temp))

/content/drive/My Drive/aiapp/class_3_clover_data_07/
['four_leaf_clover', 'three_leaf_clover', 'autumn_leaves']


#ファイル連番振り直し

#ファイルの読込フォルダと書込みフォルダ


In [0]:
    #①icrawler 保存先path
    #path_app="/content/drive/My Drive/aiapp/"
    path_app="/content/drive/My Drive/aiapp/"

    #original_dir=path_app+"☆_data_0☆/"
    path_original = path_app+"class_3_clover_data_0☆/"
    #path_original = path_app+"class_3_Flicer_data_01/"
    original_dir = path_original
    print ("original_dir=path_original=")
    print (original_dir)

original_dir=path_original=
/content/drive/My Drive/aiapp/class_3_clover_data_04/


In [0]:

# 連番リネーム 500番以降に振りなおす
import glob, os # --- (*1)
# ファイル列挙 --- (*2)

#data_dir='/content/drive/My Drive/aiapp/class_3_clover_data_03/four_leaf_clover/'
#data_dir='/content/drive/My Drive/aiapp/class_3_clover_data_03/three_leaf_clover/'
data_dir='/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/'

files = glob.glob(data_dir+"*.jpg")
# 繰り返しリネーム --- (*3)
i = 1
for old_name in files:
    # 数値を6桁(000001)形式にそろえる --- (*4)
    zero_i = "{0:06d}".format(i)
    # ファイル名を作る
    new_name = data_dir+zero_i + ".jpg"
    # 改名する --- (*5)
    os.rename(old_name, new_name)
    # 状況を報告
    print(old_name + "→" + new_name)
    # 連番のため数値を加算 --- (*6)
    i += 1


/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000001.jpg→/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000001.jpg
/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000002.jpg→/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000002.jpg
/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000003.jpg→/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000003.jpg
/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000004.jpg→/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000004.jpg
/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000005.jpg→/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000005.jpg
/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000006.jpg→/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leaves/000006.jpg
/content/drive/My Drive/aiapp/class_3_clover_data_03/autumn_leav